In [1]:
import json
import re
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from ast import literal_eval

import os
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import xlearn as xl
import tensorflow.keras as keras
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from surprise import dump

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 110)

In [2]:
class ReviewData():
    def __init__(self):
        self.review_df = pd.DataFrame()
        self.restaurant_df = pd.DataFrame()
        self.restaurant_whole_df = pd.DataFrame()
        self.load_data()
    
    def load_data(self):
        df = pd.read_csv('review.csv',index_col='Unnamed: 0', converters={'categories': literal_eval})
        self.review_df = df
        df = pd.read_csv('restaurant.csv',index_col='Unnamed: 0')
        self.restaurant_df = df
        df = pd.read_csv('restaurant_whole.csv',index_col='Unnamed: 0')
        self.restaurant_whole_df = df

In [3]:
class PredictXLearn():
    def __init__(self, _data_object):
        self._user_id = ''
        self._xreviews = _data_object.review_df
        self._xrestaurants = _data_object.restaurant_df
        self._predict_df = pd.DataFrame()
        
        self.field_dict = {'Restaurant': '0', 'User': '1', 'Category': '2'}
        self.mapping_dict = {'useful': 'Restaurant', 'funny': 'Restaurant', 'cool':'Restaurant','review_count':'Restaurant',
                        'user_review_count': 'User', 'user_useful': 'User', 'user_funny':'User','user_cool':'User','fans':'User','average_stars':'User',
                        'categories': 'Category'}
        self.col_len_dict = {'useful':10,'funny':10,'cool':10,'review_count':10,'categories':88,'user_review_count':10,'user_useful':10,'user_funny':10,'user_cool':10,'fans':10,'average_stars':1}
        self.col_accum_index_dict = {}
        cumulative = 0
        for key, value in self.col_len_dict.items():
            self.col_accum_index_dict[key] = cumulative
            cumulative = cumulative + value
            
    def build_predict_df(self, _user_id):
        predict_df = self._xreviews[['useful','funny','cool','review_count','bid']]
        predict_df = predict_df.drop_duplicates(subset=['bid'])
        predict_user_df = self._xreviews[self._xreviews['uid']==self._user_id][['stars','user_review_count','user_useful','user_funny','user_cool','fans','average_stars','uid']]
        predict_user_df = predict_user_df.drop_duplicates(subset=['uid'])
        predict_df['key'] = 0
        predict_user_df['key'] = 0
        predict_df = predict_df.merge(predict_user_df, how='outer')
        self._predict_df = predict_df
        
    def build_libffm(self, _user_id):
        self._user_id = _user_id
        self.build_predict_df(_user_id)
        with open('xlearn_dataset/predict_'+self._user_id+'.txt', 'w') as txt_file:
            predict_df = self._predict_df
            for idx, row in predict_df.iterrows():
                vec = []
                label = row['stars']
                vec.append(str(label))
                row = row.drop(labels=['stars','bid','uid','key'])
                for key, value in row.items():
                    if key == 'average_stars':
                        col_idx = self.col_accum_index_dict[key]
                        out_val = value
                        vec.append(self.field_dict[self.mapping_dict[key]] + ":" + str(col_idx) + ":" + str(out_val))
                    elif key == 'categories':
                        col_idx = self.col_accum_index_dict[key] - 1
                        out_val = 1
                        [vec.append(self.field_dict[self.mapping_dict[key]] + ":" + str(col_idx+n) + ":" + str(out_val)) for n in value if n >0]
                    else:
                        col_idx = self.col_accum_index_dict[key] + (int(value) - 1)
                        out_val = 1
                        vec.append(self.field_dict[self.mapping_dict[key]] + ":" + str(col_idx) + ":" + str(out_val))
                txt_file.write("%s\n" % " ".join(vec))
                
    def predict(self, num=10):
        fm_model = xl.create_fm()
        # Prediction task
        predict_path = 'xlearn_dataset/predict_'+self._user_id+'.txt'
        fm_model.setTest(predict_path)  # Set the path of test dataset
        # Start to predict
        # The output result will be stored in output.txt
        fm_model.predict('final_model/xLearn_v0.1.out', 'xlearn_dataset/output_'+self._user_id+'.txt')
        result = pd.read_csv('xlearn_dataset/output_'+self._user_id+'.txt', header=None, names=['final_stars'])        
        result = pd.concat([self._xrestaurants,result], axis=1).sort_values('final_stars', ascending=False)[:num]

        return result
    
    def simple_predict(self):
        fm_model = xl.create_fm()
        # Prediction task
        predict_path = 'xlearn_dataset/predict_'+self._user_id+'.txt'
        fm_model.setTest(predict_path)  # Set the path of test dataset
        # Start to predict
        # The output result will be stored in output.txt
        fm_model.predict('final_model/xLearn_v0.1.out', 'xlearn_dataset/output_'+self._user_id+'.txt')
        result = pd.read_csv('xlearn_dataset/output_'+self._user_id+'.txt', header=None, names=['xlearn_stars'])        
        result = pd.concat([self._xrestaurants['business_id'],result], axis=1)

        return result

In [90]:
a = ReviewData()
b = PredictXLearn(a)

In [92]:
b.build_libffm('F_N4NyoMJWzBYFspeYfH3A')
c = b.predict()
c

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.GoodForKids,attributes.RestaurantsReservations,attributes.Caters,attributes.NoiseLevel,attributes.RestaurantsTableService,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange2,attributes.OutdoorSeating,attributes.BikeParking,attributes.HasTV,attributes.WiFi,attributes.Alcohol,attributes.RestaurantsAttire,attributes.RestaurantsGoodForGroups,attributes.RestaurantsDelivery,attributes.GoodForMeal.dessert,attributes.GoodForMeal.latenight,attributes.GoodForMeal.lunch,attributes.GoodForMeal.dinner,attributes.GoodForMeal.brunch,attributes.GoodForMeal.breakfast,attributes.BusinessParking.garage,attributes.BusinessParking.street,attributes.BusinessParking.validated,attributes.BusinessParking.lot,attributes.BusinessParking.valet,attributes.Ambience.romantic,attributes.Ambience.intimate,attributes.Ambience.classy,attributes.Ambience.hipster,attributes.Ambience.divey,attributes.Ambience.touristy,attributes.Ambience.trendy,attributes.Ambience.upscale,attributes.Ambience.casual,attributes.BusinessAcceptsCreditCards,attributes.BusinessAcceptsBitcoin,attributes.ByAppointmentOnly,attributes.AcceptsInsurance,attributes.GoodForDancing,attributes.CoatCheck,attributes.HappyHour,attributes.Music.dj,attributes.Music.background_music,attributes.Music.no_music,attributes.Music.jukebox,attributes.Music.live,attributes.Music.video,attributes.Music.karaoke,attributes.BestNights.monday,attributes.BestNights.tuesday,attributes.BestNights.friday,attributes.BestNights.wednesday,attributes.BestNights.thursday,attributes.BestNights.sunday,attributes.BestNights.saturday,attributes.Ambience,attributes.BusinessParking,attributes.DogsAllowed,attributes.BYOBCorkage,attributes.DriveThru,attributes.Smoking,attributes.AgesAllowed,attributes.Corkage,attributes.BYOB,attributes.GoodForMeal,attributes.Music,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.gluten-free,attributes.DietaryRestrictions.vegan,attributes.DietaryRestrictions.kosher,attributes.DietaryRestrictions.halal,attributes.DietaryRestrictions.soy-free,attributes.DietaryRestrictions.vegetarian,attributes.BestNights,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService,final_stars
2188,JFNv5sKeKAf1ZVPnfbzpXw,East West Pizzeria,5025 S Fort Apache Rd,Las Vegas,NV,89148.0,36.097968,-115.297796,4.0,153,1,"Pizza,Restaurants,Food",True,False,True,quiet,NaN,True,2,False,True,True,free,none,casual,True,True,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.78508
4119,odQGs2Rv0nC0fHh4Gfp2iA,Ramen Noodle,"3350 Novat St, Ste 135",Las Vegas,NV,89129.0,36.221074,-115.328577,4.5,153,1,"Ramen,Sushi Bars,Restaurants,Noodles",True,True,NaN,quiet,True,True,2,False,True,True,no,none,casual,True,True,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.78508
6349,Ih5Q1cNDLchhi1FqqTwrag,Turmeric Flavors of India,700 E Fremont St,Las Vegas,NV,89101.0,36.168374,-115.138119,4.5,153,1,"Restaurants,Bars",True,True,True,average,True,True,2,True,True,True,free,full_bar,casual,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,True,True,NaN,NaN,NaN,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,True,NaN,NaN,False,NaN,NaN,outdoor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.78508
6033,2Wl4o7l54I-EtxtaDpOTzA,Pub 365,255 E Flamingo Rd,Las Vegas,NV,8916

In [4]:
class PredictDeepFM():
    def __init__(self, _data_object, n_max_category = 12, n_category = 88):
        self._user_id = ''
        self._xreviews = _data_object.review_df
        self._xrestaurants = _data_object.restaurant_df
        self._xdf2xy = pd.DataFrame()
        self._predict_df = pd.DataFrame()
        self._n_max_category = n_max_category
        self._n_category = n_category
        self._deep_fm_model = load_model('final_model/deepFM_v0.1.h5')
    
    def build_predict_df(self, _user_id):
        final_k = pd.DataFrame({'business_id':self._xreviews['business_id'].unique()})
        final_review = final_k.merge(self._xreviews[['business_id','name','bid','restaurant_stars','review_count','categories']], how='left',on='business_id')
        final_review = final_review.drop_duplicates(subset=['business_id'])
        final_user = self._xreviews[self._xreviews['uid']==_user_id][['user_id','user_review_count','user_useful','user_funny','user_cool','fans','average_stars','stars']]
        final_user = final_user.drop_duplicates(subset=['user_id'])
        final_review['key'] = 0
        final_user['key'] = 0
        final_df = final_review.merge(final_user, how='outer').drop(['key'],axis=1)
        self._predict_df = final_df
        
    def build_df2xy(self, _user_id):
        self._user_id = _user_id
        self.build_predict_df(_user_id)
        self._predict_df = [self._predict_df.restaurant_stars.values,
                       self._predict_df.review_count.values,
                       self._predict_df.average_stars.values,
                       self._predict_df.user_id.values, 
                       self._predict_df.business_id.values, 
                       np.concatenate(self._predict_df.categories.values).reshape(-1,self._n_max_category)]
        
    def predict(self, num=10):
        result = pd.DataFrame(self._deep_fm_model.predict(self._predict_df), columns=['final_stars'])
        result = pd.concat([self._xrestaurants,result], axis=1).sort_values('final_stars', ascending=False)[:num]
        return result
    
    def simple_predict(self):
        result = pd.DataFrame(self._deep_fm_model.predict(self._predict_df), columns=['deepfm_stars'])
        result = pd.concat([self._xrestaurants['business_id'],result], axis=1)
        return result

In [94]:
d = PredictDeepFM(a)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/Users/hayley/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:891: UserWarning: tensorflow.keras.layers is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


In [95]:
d.build_df2xy('F_N4NyoMJWzBYFspeYfH3A')
e = d.predict()
e

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.GoodForKids,attributes.RestaurantsReservations,attributes.Caters,attributes.NoiseLevel,attributes.RestaurantsTableService,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange2,attributes.OutdoorSeating,attributes.BikeParking,attributes.HasTV,attributes.WiFi,attributes.Alcohol,attributes.RestaurantsAttire,attributes.RestaurantsGoodForGroups,attributes.RestaurantsDelivery,attributes.GoodForMeal.dessert,attributes.GoodForMeal.latenight,attributes.GoodForMeal.lunch,attributes.GoodForMeal.dinner,attributes.GoodForMeal.brunch,attributes.GoodForMeal.breakfast,attributes.BusinessParking.garage,attributes.BusinessParking.street,attributes.BusinessParking.validated,attributes.BusinessParking.lot,attributes.BusinessParking.valet,attributes.Ambience.romantic,attributes.Ambience.intimate,attributes.Ambience.classy,attributes.Ambience.hipster,attributes.Ambience.divey,attributes.Ambience.touristy,attributes.Ambience.trendy,attributes.Ambience.upscale,attributes.Ambience.casual,attributes.BusinessAcceptsCreditCards,attributes.BusinessAcceptsBitcoin,attributes.ByAppointmentOnly,attributes.AcceptsInsurance,attributes.GoodForDancing,attributes.CoatCheck,attributes.HappyHour,attributes.Music.dj,attributes.Music.background_music,attributes.Music.no_music,attributes.Music.jukebox,attributes.Music.live,attributes.Music.video,attributes.Music.karaoke,attributes.BestNights.monday,attributes.BestNights.tuesday,attributes.BestNights.friday,attributes.BestNights.wednesday,attributes.BestNights.thursday,attributes.BestNights.sunday,attributes.BestNights.saturday,attributes.Ambience,attributes.BusinessParking,attributes.DogsAllowed,attributes.BYOBCorkage,attributes.DriveThru,attributes.Smoking,attributes.AgesAllowed,attributes.Corkage,attributes.BYOB,attributes.GoodForMeal,attributes.Music,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.gluten-free,attributes.DietaryRestrictions.vegan,attributes.DietaryRestrictions.kosher,attributes.DietaryRestrictions.halal,attributes.DietaryRestrictions.soy-free,attributes.DietaryRestrictions.vegetarian,attributes.BestNights,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService,final_stars
3334,wMqu9LseUcq7bqWWY-3oTQ,Jamba Juice,10830 W Charleston Blvd,Las Vegas,NV,89135.0,36.161244,-115.331600,3.5,27,1,"Food,Juice Bars & Smoothies",NaN,NaN,True,NaN,NaN,True,2,NaN,True,NaN,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.011297
7028,kpC__sWtWkLdSOI2xxdirg,Paddy's Pub,"4160 S Pecos Rd, Ste 1",Las Vegas,NV,89121.0,36.113653,-115.100011,3.0,6,1,"Bars,Pubs",NaN,False,NaN,average,NaN,NaN,2,False,False,True,free,full_bar,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False,NaN,NaN,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.009379
5881,yUzII0aClaBc7K7kKY6bJQ,Village Meat & Wine,"5025 S Eastern Ave, Ste 23",Las Vegas,NV,89119.0,36.098107,-115.119846,4.5,41,1,"Beer,Wine & Spirits,Food,Specialty Food",NaN,NaN,NaN,NaN,NaN,False,2,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.897320
2854,zpXTbidP23E8Yaxyh8c4HA,Port of Subs,4412 N Rancho Dr,Las Vegas,NV,89130.0,36.239972,-115.230348,4.0,8,0,"Restaurants,Sandwiches",True,False,True,quiet,NaN,True,1,False,NaN,False,free,none,ca

In [5]:
class PredictSurpriseSVD():
    def __init__(self, _data_object):
        self._user_id = ''
        self._xrestaurants_whole = _data_object.restaurant_whole_df
        self._predict_df = pd.DataFrame()
        _, self._surprise_svd_model = dump.load('final_model/surpriseSVD_v0.1')
        
    def build_predict_df(self, _user_id):
        predict_df = self._xrestaurants_whole[['business_id']]
        predict_df['user_id']= _user_id
        self._predict_df = predict_df
        
    def predict(self, num=10):
        for index, row in self._predict_df.iterrows():
            prediction = self._surprise_svd_model.predict(uid=row["user_id"],iid=row["business_id"])
            self._predict_df.at[index, 'final_stars'] = prediction.est    
        
        result = pd.concat([self._xrestaurants_whole, self._predict_df], axis=1).sort_values('final_stars', ascending=False)[:num]
        return result
    
    def simple_predict(self):
        for index, row in self._predict_df.iterrows():
            prediction = self._surprise_svd_model.predict(uid=row["user_id"],iid=row["business_id"])
            self._predict_df.at[index, 'svd_stars'] = prediction.est    
        
        result = self._predict_df[['business_id','svd_stars']]
        return result

In [97]:
a = ReviewData()

/Users/hayley/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (19,51,77,87,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [98]:
g = PredictSurpriseSVD(a)

In [99]:
g.build_predict_df('F_N4NyoMJWzBYFspeYfH3A')
h = g.simple_predict()
h

/Users/hayley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/hayley/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/hayley/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,business_id,svd_stars
17,PZ-LZzSlhSe9utkQYU8pFg,4.021082
22,KWywu2tTEPWmR9JnBc0WyQ,3.683928
25,tstimHoMcYbkSC4eBA1wEg,4.141700
75,kANF0dbeoW34s2vwh6Umfw,2.032764
101,YK5eFV8uS1bCmp8GeiQ-Qg,3.203250
111,X8mtoSxY8whtmbDlj0D3Aw,2.707935
128,sKhDrZFCJqfRNylkHrIDsQ,3.465839
135,bJP4l_BGq2CudEu0m-wNjg,2.583203
173,v7ZFEEqJBg_r8NCwHRP_2A,3.497890
174,6fPQJq4f_yiq1NHn0fd11Q,3.462543


In [9]:
class Enssenble():
    def __init__(self, _data_obj):
        self._data_object = _data_obj
        self._xlearn = PredictXLearn(self._data_object)
        self._deepfm = PredictDeepFM(self._data_object)
        #self._surpriseSvd = PredictSurpriseSVD(self._data_object)
        
    def predict(self, _user_id, _num = 10):
        self._xlearn.build_libffm(_user_id)
        self._deepfm.build_df2xy(_user_id)        
        self._surpriseSvd.build_predict_df(_user_id)
        
        xlearn = self._xlearn.simple_predict()
        deepfm = self._deepfm.simple_predict()
        surpriseSvd = self._surpriseSvd.simple_predict()
            
        result = self._data_object.restaurant_whole_df
        result = result.merge(xlearn, on='business_id', how='left').merge(deepfm, on='business_id', how='left').merge(surpriseSvd, on='business_id', how='left')
        
        scaler = MinMaxScaler(feature_range=(1,10))
        result['deepfm_stars'] = scaler.fit_transform(result[['deepfm_stars']])
        result['xlearn_stars'] = scaler.fit_transform(result[['xlearn_stars']])
        #result['svd_stars'] = scaler.fit_transform(result[['svd_stars']])
        is_in = (_user_id in self._data_object.review_df['uid'].values)
        result['final_stars'] = result.apply(lambda x : x['svd_stars']*0.9 if (not is_in) or (np.isnan(x['deepfm_stars'])) or (np.isnan(x['xlearn_stars'])) else x['deepfm_stars']*0.4 + x['xlearn_stars']*0.3 + x['svd_stars']*0.3, axis=1)
        result = result.sort_values('final_stars', ascending = False)[:_num]
        
        #result = result.sort_values('final_stars', ascending = False)
        
        return result        

In [236]:
print(not True)

False


In [228]:
b.simple_predict().describe()

,xlearn_stars
count,8655.000000
mean,3.977247
std,0.169283
min,3.014860
25%,3.898790
50%,3.991440
75%,4.061450
max,4.785080


In [137]:
d.simple_predict().describe()

,deepfm_stars
count,8655.000000
mean,3.003921
std,1.032167
min,-1.253405
25%,2.439859
50%,2.972907
75%,3.725081
max,6.011297


In [101]:
g.simple_predict().describe()

,svd_stars
count,8864.000000
mean,3.299807
std,0.648164
min,1.000000
25%,2.852181
50%,3.350921
75%,3.791549
max,4.876780


In [7]:
a = ReviewData()
#f = Enssenble(a)

/Users/necronia/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (19,51,77,87,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
# reveiw 1??? 
#g = f.predict('gEBRecs8QUw9kxlbqIN_sA')
# review 1643
f = Enssenble(a)
#g = f.predict('PKEzKWv_FktMm2mGPjwd0Q')

/Users/necronia/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:891: UserWarning: tensorflow.keras.layers is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


In [31]:
f._data_object.restaurant_df[f._data_object.restaurant_df['business_id']=='PZ-LZzSlhSe9utkQYU8pFg'].iloc[0].to_dict()

{'business_id': 'PZ-LZzSlhSe9utkQYU8pFg',
 'name': "Carluccio's Tivoli Gardens",
 'address': '1775 E Tropicana Ave, Ste 29',
 'city': 'Las Vegas',
 'state': 'NV',
 'postal_code': 89119.0,
 'latitude': 36.10001629999999,
 'longitude': -115.12852849999999,
 'stars': 4.0,
 'review_count': 40,
 'is_open': 0,
 'categories': 'Restaurants',
 'attributes.GoodForKids': 'True',
 'attributes.RestaurantsReservations': 'True',
 'attributes.Caters': 'True',
 'attributes.NoiseLevel': 'quiet',
 'attributes.RestaurantsTableService': nan,
 'attributes.RestaurantsTakeOut': 'True',
 'attributes.RestaurantsPriceRange2': '2',
 'attributes.OutdoorSeating': 'False',
 'attributes.BikeParking': nan,
 'attributes.HasTV': 'False',
 'attributes.WiFi': 'no',
 'attributes.Alcohol': 'full_bar',
 'attributes.RestaurantsAttire': 'casual',
 'attributes.RestaurantsGoodForGroups': 'True',
 'attributes.RestaurantsDelivery': 'False',
 'attributes.GoodForMeal.dessert': nan,
 'attributes.GoodForMeal.latenight': nan,
 'attribu

In [11]:
#g = f.predict('UYcmGbelzRa0Q6JqzLoguw')
g = f.predict('rt3PC7WCgCKsoufmQJELfw') #8.987600
#g = f.predict('tH0uKD-vNwMoEc3Xk3Cbdg') #8.892691
g

AttributeError: 'Enssenble' object has no attribute '_surpriseSvd'

In [104]:
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = RobustScaler()
deepfm_df['deepfm_final_score'] = scaler.fit_transform(deepfm_df[['deepfm_final_score']])
xlearn_df['xlearn_final_score'] = scaler.fit_transform(xlearn_df[['xlearn_final_score']])
final_df = final_df.merge(deepfm_df,on='business_id',how='left').merge(xlearn_df, on='business_id',how='left')
final_df['final_score'] = final_df['deepfm_final_score'] * 0.3 + final_df['xlearn_final_score'] * 0.7
final_df = final_df.sort_values('final_score',ascending=False)

NameError: name 'deepfm_df' is not defined

In [ ]:
'F_N4NyoMJWzBYFspeYfH3A' in a.review_df['uid'].values